# Updated code using reward_function_3 with a model accuracy 70% +

In [4]:
import pandas as pd
df = pd.read_csv('file1.csv')
alpha = 0.1
previousValues = []

def rewardget(score, previousValues):
    previousValues_mean = sum(previousValues[-10:]) / len(previousValues[-10:]) if previousValues else 0
    if score >= 0 and score <= 19:
        reward = 50 - (previousValues_mean / 10)
    elif score >= 20 and score <= 39:
        reward = 40 - (previousValues_mean / 10)
    elif score >= 40 and score <= 59:
        reward = 30 - (previousValues_mean / 10)
    elif score >= 60 and score <= 79:
        reward = 20 - (previousValues_mean / 10)
    elif score >= 80 and score <= 99:
        reward = 10 - (previousValues_mean / 10)
    elif score == 100:
        reward = 0 - (previousValues_mean / 10)
    elif score >= 101 and score <= 120:
        reward = -10 - (previousValues_mean / 10)
    elif score >= 121 and score <= 140:
        reward = -20 - (previousValues_mean / 10)
    elif score >= 141 and score <= 160:
        reward = -30 - (previousValues_mean / 10)
    elif score >= 161 and score <= 180:
        reward = -40 - (previousValues_mean / 10)
    elif score >= 181 and score <= 200:
        reward = -50 - (previousValues_mean / 10)
    return reward

def valuefind(reward, previousValue, prev_score, current_score):
    if abs(current_score - prev_score) > 100:
        currentValue = 2 * reward + alpha * previousValue
    else:
        currentValue = reward + alpha * previousValue
    return currentValue

def send_notification(message):
    return message

currentValues = []
previousValue = 0
prev_score = df.loc[0, 'score']  # Initialize prev_score with the first score in the dataframe

notification_messages = []

for i in range(len(df)):
    score = df.loc[i, 'score']
    reward = rewardget(score, previousValues)
    currentValue = valuefind(reward, previousValue, prev_score, score)
    currentValues.append(currentValue)
    previousValues.append(currentValue)
    previousValue = currentValue
    prev_score = score
    
    # Check if an attacker query has occurred or not
    if df.loc[i, 'attack'] == 0:
        # Reset previousValues and previousValue
        previousValues = []
        previousValue = 0

    if currentValue < 0:
        message = "Please be cautious."
        notification_messages.append(send_notification(message))
    else:
        message = "Safe Query"
        notification_messages.append(send_notification(message))

df['currentValues'] = currentValues
df['notification_messages'] = notification_messages
df.head(20)

,Unnamed: 0,query,score,attack,currentValues,notification_messages
0,0,Q1,73,0,20.000000,Safe Query
1,1,Q2,38,0,40.000000,Safe Query
2,2,Q3,128,1,-20.000000,Please be cautious.
3,3,Q4,179,1,-40.000000,Please be cautious.
4,4,Q5,173,1,-41.000000,Please be cautious.
5,5,Q6,91,0,9.266667,Safe Query
6,6,Q7,182,1,-50.000000,Please be cautious.
7,7,Q8,200,1,-50.000000,Please be cautious.
8,8,Q9,15,0,105.000000,Safe Query
9,9,Q10,84,0,10.000000,Safe Query


In [5]:
df.head(50)

,Unnamed: 0,query,score,attack,currentValues,notification_messages
0,0,Q1,73,0,20.000000,Safe Query
1,1,Q2,38,0,40.000000,Safe Query
2,2,Q3,128,1,-20.000000,Please be cautious.
3,3,Q4,179,1,-40.000000,Please be cautious.
4,4,Q5,173,1,-41.000000,Please be cautious.
5,5,Q6,91,0,9.266667,Safe Query
6,6,Q7,182,1,-50.000000,Please be cautious.
7,7,Q8,200,1,-50.000000,Please be cautious.
8,8,Q9,15,0,105.000000,Safe Query
9,9,Q10,84,0,10.000000,Safe Query


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


# Separate input features and output
X = df[['score', 'attack']]
y = df['currentValues']

# Split the dataframe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

model = LinearRegression()
model.fit(X_train, y_train)

# predictions
y_pred = model.predict(X_test)

# Calculate accuracy
mse = mean_squared_error(y_test, y_pred)
accuracy = 1 - (mse / y_test.var())
print("Accuracy:", accuracy)

Accuracy: 0.8776731516713059
